In [ ]:
# -*- coding: utf-8 -*-
"""
LSTM模型用于共享单车租赁预测
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

# 解决画图中文显示问题
# plt.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['axes.unicode_minus'] = False

# 参数设置
look_back = 96  # 输入的历史时间步
epochs = 50  # 迭代次数
batch_size = 64  # 批量大小
input_dim = 9  # 输入特征数
hidden_dim = 64  # 隐藏层神经元数
num_layers = 2  # LSTM层数
dropout = 0.2  # Dropout
learn_rate = 0.0005  # 学习率
T = 240

# 加载数据
# train_data = pd.read_csv('train_data.csv')
# test_data = pd.read_csv('test_data.csv')

data_path = r'../data/'
train_data = pd.read_csv(data_path + 'train_data.csv')
test_data = pd.read_csv(data_path + 'test_data.csv')

# 选择特征和目标列
# features = ['season', 'holiday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
features = [  'temp', 'atemp', 'hum', 'windspeed']
target = 'cnt'

train_data[features].fillna(0, inplace=True)
train_data[target].fillna(0, inplace=True)


# 数据预处理：归一化
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

train_data[features] = scaler_features.fit_transform(train_data[features])
train_data[target] = scaler_target.fit_transform(train_data[[target]])
test_data[features] = scaler_features.transform(test_data[features])
test_data[target] = scaler_target.transform(test_data[[target]])

train_data[features] = train_data[features].fillna(train_data[features].mean())
test_data[features] = test_data[features].fillna(test_data[features].mean())

features = ['season', 'cnt', 'holiday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']


# 构建时间序列数据
def create_sequences(data, features, target, look_back):
    X, y = [], []
    for i in range(0, len(data) - T-look_back, look_back):
        X.append(data[features].iloc[i:i + look_back].values)
        y.append(data[target].iloc[i + look_back:i + look_back + T].values)
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, features, target, look_back)
X_test, y_test = create_sequences(test_data, features, target, look_back)

# 转换为 PyTorch Tensor 
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)#.unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)#.unsqueeze(1)

# 数据加载器
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_dim, output_dim*2)
        self.fc2 = nn.Linear(output_dim*2, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)  # 取最后时间步的输出
        return out

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=T, num_layers=num_layers, dropout=dropout)
model = model.to(device)

# 初始化权重
def init_weights(m):
    if isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

model.apply(init_weights)

# 损失函数与优化器
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=learn_rate)

# 梯度裁剪
def clip_gradients(model, max_norm=1.0):
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

# 训练
train_losses = []
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        if torch.isnan(loss):
            raise ValueError("Loss became NaN during training.")
        
        loss.backward()
        clip_gradients(model)
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}")

# 测试
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(y_batch.cpu().numpy())

# 反归一化
predictions = scaler_target.inverse_transform(np.array(predictions).reshape(-1, 1))
actuals = scaler_target.inverse_transform(np.array(actuals).reshape(-1, 1))

# predictions = scaler_target.inverse_transform(np.array(predictions).reshape(-1, 1))
# actuals = scaler_target.inverse_transform(np.array(actuals).reshape(-1, 1))

# 评估
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}")

# 可视化结果


/tmp/ipykernel_3157831/2587082086.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[features].fillna(0, inplace=True)


Epoch [1/500], Loss: 0.0635
Epoch [2/500], Loss: 0.0501
Epoch [3/500], Loss: 0.0387
Epoch [4/500], Loss: 0.0330
Epoch [5/500], Loss: 0.0327
Epoch [6/500], Loss: 0.0316
Epoch [7/500], Loss: 0.0308
Epoch [8/500], Loss: 0.0324
Epoch [9/500], Loss: 0.0310
Epoch [10/500], Loss: 0.0306
Epoch [11/500], Loss: 0.0303
Epoch [12/500], Loss: 0.0307
Epoch [13/500], Loss: 0.0297
Epoch [14/500], Loss: 0.0299
Epoch [15/500], Loss: 0.0295
Epoch [16/500], Loss: 0.0298
Epoch [17/500], Loss: 0.0296
Epoch [18/500], Loss: 0.0291
Epoch [19/500], Loss: 0.0281
Epoch [20/500], Loss: 0.0275
Epoch [21/500], Loss: 0.0278
Epoch [22/500], Loss: 0.0263
Epoch [23/500], Loss: 0.0257
Epoch [24/500], Loss: 0.0249
Epoch [25/500], Loss: 0.0251
Epoch [26/500], Loss: 0.0242
Epoch [27/500], Loss: 0.0235
Epoch [28/500], Loss: 0.0222
Epoch [29/500], Loss: 0.0216
Epoch [30/500], Loss: 0.0210
Epoch [31/500], Loss: 0.0192
Epoch [32/500], Loss: 0.0188
Epoch [33/500], Loss: 0.0176
Epoch [34/500], Loss: 0.0166
Epoch [35/500], Loss: 0

In [11]:
y_test

array([array([0.80348004, 0.39303992, 0.26612078, 0.3111566 , 0.4790174 ,
              0.43398158, 0.40941658, 0.48106448, 0.64892528, 0.92118731,
              0.77891505, 0.51177073, 0.38075742, 0.27328557, 0.27021494,
              0.17502559, 0.19549642, 0.14431934, 0.07676561, 0.05731832,
              0.01125896, 0.01023541, 0.03070624, 0.08597748, 0.21084954,
              0.40429887, 0.55168884, 0.66223132, 0.7604913 , 0.72671443,
              0.58955988, 0.6345957 , 0.67451382, 0.62436029, 0.50665302,
              0.34902764, 0.25281474, 0.22006141, 0.18935517, 0.18321392,
              0.12998976, 0.11156602, 0.07574207, 0.02251791, 0.01125896,
              0.01023541, 0.02354145, 0.03582395, 0.06345957, 0.08802456,
              0.18935517, 0.34390993, 0.40122825, 0.32753327, 0.32139202,
              0.31320368, 0.3408393 , 0.16069601, 0.10849539, 0.11668373,
              0.11873081, 0.11566018, 0.08085977, 0.08188332, 0.05220061,
              0.03787103, 0.01535312, 

In [3]:


best_i = -1
mse = float('inf')
for i in range(len(actuals)-96-T):
    tmp = mean_absolute_error(actuals[i+96:i+96+T], predictions[i:i+T])
    if tmp < mse:
        mse = tmp
        best_i = i


i = best_i
y1 = np.concatenate((actuals[i:i+96], actuals[i+96:i+96+T]))
y2 = predictions[i:i+T]

# 计算两条曲线的 x 轴范围
x1 = np.arange(len(y1))  # x1: [0, 1, 2, 3]
x2 = np.arange(len(y1) - len(y2), len(y1))  # x2: [2, 3]
plt.figure(figsize=(16, 8), dpi=600)
# 绘制曲线
plt.plot(x1, y1, label="True")  # 第一条曲线
plt.plot(x2, y2, label="Prediction")  # 第二条曲线

# 添加图例和标题
plt.legend()

# plt.xlabel("X")
# plt.ylabel("Y")

# 显示图形
plt.show()


NameError: name 'actuals' is not defined

In [15]:
mse

91.00915

In [2]:
i

1900